In [2]:
from faker import Faker
import random
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv(".env")

fake = Faker()

# Partie a - Générer les requêtes pour insérer 100 utilisateurs
users_insert_queries = []
user_created_dates = {}
user_verified_dates = {}


for _ in range(100):
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = fake.unique.email()
    user_name = fake.unique.user_name()
    while len(user_name) <= 8:
        user_name = fake.unique.user_name()
    password = fake.password()
    while len(password) <= 8:
        password = fake.password()
    user_created_at = fake.date_time_between_dates(datetime_start=datetime(2023, 1, 1), datetime_end=datetime(2023, 6, 30))
    user_created_dates[_ + 1] = user_created_at  # Storing user creation dates in a dictionary
    user_query = f"INSERT INTO user_table (firstname, lastname, email, username, password, created_at) VALUES ('{first_name}', '{last_name}', '{email}', '{user_name}', '{password}', '{user_created_at.strftime('%Y-%m-%d %H:%M:%S')}');"
    users_insert_queries.append(user_query)

# Partie b - Générer les requêtes pour insérer 85 vérifications d'emails
email_verifications_insert_queries = []

for user_id, user_created_at in user_created_dates.items():
    # Générer la date de vérification d'email après la création du compte
    verified_at = fake.date_time_between_dates(datetime_start=user_created_at, datetime_end=datetime(2023, 6, 30))
    user_verified_dates[user_id] = verified_at  # Utiliser user_id comme clé pour le dictionnaire
    # Créer la requête SQL
    verification_query = f"INSERT INTO user_email_verification_table (user_id, verified_at) VALUES ({user_id}, '{verified_at.strftime('%Y-%m-%d %H:%M:%S')}');"
    email_verifications_insert_queries.append(verification_query)

usr_id_for_verif = random.sample(range(1, 101), 85)

selected_email_verifications = [
    query for idx, query in enumerate(email_verifications_insert_queries, 1)
    if idx in usr_id_for_verif
]

# Partie c - Générer les requêtes pour insérer 1000 sessions
sessions_insert_queries = []

# Filtrer les utilisateurs ayant une vérification d'email
verified_users = usr_id_for_verif
for _ in range(1000):  # Générer 1000 sessions au total
    user_id = random.choice(verified_users)

    # Vérifier si l'utilisateur a une date de vérification d'email
    if user_id in user_verified_dates:
        user_verified_at = user_verified_dates[user_id]

        connected_at = fake.date_time_between_dates(
            datetime_start=user_verified_at,
            datetime_end=datetime(2023, 6, 30)
        )

        # Créer la requête SQL
        session_query = f"INSERT INTO session_table (user_id, connected_at) VALUES ({user_id}, '{connected_at.strftime('%Y-%m-%d %H:%M:%S')}');"
        sessions_insert_queries.append(session_query)

# Écrire les requêtes SQL générées dans un fichier
with open('part_1_2.sql', 'w') as f:
    for user in users_insert_queries:
        f.write(user + '\n')
    f.write('\n'.join(selected_email_verifications))
    f.write('\n'.join(sessions_insert_queries))


